# TCG Shadowbox Generator - クイックスタート

このノートブックでは、TCGカードのイラストを深度推定とクラスタリングで階層化し、
インタラクティブな3Dシャドーボックスとして表示する方法を説明します。

## 目次
1. セットアップ
2. 画像の読み込み
3. イラスト領域の特定
4. 深度推定とクラスタリング
5. 2D可視化
6. 3Dシャドーボックス表示

## 1. セットアップ

必要なモジュールをインポートします。

In [ ]:
# 基本的なインポート
import matplotlib.pyplot as plt
from PIL import Image

# shadowboxモジュールのインポート
from shadowbox import create_pipeline
from shadowbox.config import BoundingBox, ShadowboxSettings
from shadowbox.detection import detect_illustration_region
from shadowbox.gui import ImageSelector, select_illustration_region, load_from_url
from shadowbox.visualization import (
    create_depth_heatmap,
    show_clustering_summary,
    render_shadowbox,
    RenderOptions,
)

# matplotlibの日本語フォント設定（Windows）
import matplotlib
matplotlib.rcParams['font.family'] = ['MS Gothic', 'Yu Gothic', 'Meiryo', 'sans-serif']
matplotlib.rcParams['axes.unicode_minus'] = False  # マイナス記号の文字化け対策

print("セットアップ完了！")

## 2. 画像の読み込み

画像を読み込む方法は3つあります：
1. URLから読み込み
2. ローカルファイルから読み込み
3. ディレクトリからギャラリー選択

### 方法1: URLから読み込み

In [ ]:
# URLから画像を読み込む場合
# 実際のカード画像URLに変更してください
image_url = "https://example.com/your-card-image.png"  # ← ここにURLを設定
image = load_from_url(image_url)

# または、ローカルファイルから読み込み
# from shadowbox.utils.image import load_image
# image = load_image("path/to/your/card.png")

# デモ用: ダミー画像を作成（実際の画像を使う場合はコメントアウト）
# image = Image.new("RGB", (250, 350), color=(50, 50, 50))
# pixels = image.load()
# for y in range(40, 240):
#     for x in range(20, 230):
#         r = (x * 3 + y) % 256
#         g = (x + y * 2) % 256
#         b = (x * 2 + y * 3) % 256
#         pixels[x, y] = (r, g, b)

# 画像を表示
plt.figure(figsize=(6, 8))
plt.imshow(image)
plt.title("読み込んだ画像")
plt.axis("off")
plt.show()

### 方法2: ローカルファイルから読み込み

In [ ]:
# ローカルファイルから読み込む場合
from shadowbox.utils.image import load_image
image = load_image("../data/cards/dm24ex1-044.webp")

# 画像を表示
plt.figure(figsize=(6, 8))
plt.imshow(image)
plt.title("読み込んだ画像")
plt.axis("off")
plt.show()

### 方法3: ディレクトリからギャラリー選択

In [ ]:
# ディレクトリ内の画像をギャラリー表示して選択
selector = ImageSelector("../data/cards/")
selector.show_gallery()  # サムネイル付きで一覧表示
image = selector.select(1)  # インデックスで選択

## 3. イラスト領域の特定

カード画像からイラスト部分を特定します。
自動検出と手動選択の2つの方法があります。

### 方法A: 自動検出

In [ ]:
# 自動検出でイラスト領域を特定
detection_result = detect_illustration_region(image)

print(f"検出方法: {detection_result.method}")
print(f"信頼度: {detection_result.confidence:.2f}")
print(f"バウンディングボックス: {detection_result.bbox}")

# 検出結果を可視化
fig, ax = plt.subplots(figsize=(8, 10))
ax.imshow(image)

bbox = detection_result.bbox
rect = plt.Rectangle(
    (bbox.x, bbox.y), bbox.width, bbox.height,
    linewidth=2, edgecolor='lime', facecolor='none'
)
ax.add_patch(rect)
ax.set_title(f"自動検出結果 (信頼度: {detection_result.confidence:.2f})")
ax.axis("off")
plt.show()

### 方法B: 手動選択（インタラクティブ）

In [ ]:
# 手動で領域を選択する場合（matplotlibのウィンドウが開きます）
# bbox = select_illustration_region(image)
# if bbox:
#     print(f"選択された領域: {bbox}")

# ここでは自動検出結果を使用
bbox = detection_result.bbox

## 4. 深度推定とクラスタリング

パイプラインを使用して、画像から深度を推定し、
レイヤーに分割します。

In [ ]:
# パイプラインを作成（初回実行時はモデルのダウンロードが行われます）
# use_mock_depth=True はテスト用（実際の深度推定を使用する場合はFalseに）
pipeline = create_pipeline(use_mock_depth=True)

print("パイプライン作成完了")

In [ ]:
# パイプラインを実行
# k: レイヤー数（指定しない場合は最適な値を自動探索）
# include_frame: フレーム（枠）を含めるかどうか
result = pipeline.process(
    image,
    custom_bbox=bbox,
    k=5,  # 5つのレイヤーに分割
    include_frame=True,
)

print(f"最適なレイヤー数: {result.optimal_k}")
print(f"クロップ後のサイズ: {result.cropped_image.shape}")
print(f"メッシュのレイヤー数: {result.mesh.num_layers}")
print(f"総頂点数: {result.mesh.total_vertices}")

## 5. 2D可視化

深度マップとレイヤー分割の結果を2Dで可視化します。

### 5.1 深度ヒートマップ

In [ ]:
# 深度マップをヒートマップで表示
fig, axes = create_depth_heatmap(
    result.depth_map,
    result.cropped_image,
    cmap="viridis",
    title="深度推定結果",
)
plt.show()

### 5.2 クラスタリングサマリー

In [ ]:
# クラスタリング結果の総合サマリーを表示
fig, axes = show_clustering_summary(
    result.cropped_image,
    result.depth_map,
    result.labels,
    result.centroids,
)
plt.show()

## 6. 3Dシャドーボックス表示

Vedoを使用してインタラクティブな3D表示を行います。

**操作方法:**
- マウス左ドラッグ: 回転
- マウス右ドラッグ: パン
- スクロール: ズーム

In [ ]:
# レンダリングオプションを設定
options = RenderOptions(
    background_color=(30, 30, 40),  # 暗い背景
    point_size=4.0,  # ポイントサイズ
    show_frame=True,  # フレームを表示
    window_size=(1000, 800),
    title="TCG Shadowbox Viewer",
)

# 3Dレンダリング（新しいウィンドウが開きます）
# 注: Jupyter環境では別ウィンドウで表示されます
render_shadowbox(result.mesh, options)

### 6.1 レイヤー分解表示（デバッグ用）

In [ ]:
# レイヤーを分離して表示（構造を確認しやすくする）
from shadowbox.visualization import render_layers_exploded

render_layers_exploded(result.mesh, gap_multiplier=3.0, options=options)

## 補足: 実際の深度推定を使用する場合

モック深度推定ではなく、実際のDepth Anything v2モデルを使用する場合は、
`use_mock_depth=False`を指定してください。

**注意**: 初回実行時はモデルのダウンロードに時間がかかります（約500MB）。

In [ ]:
# 実際の深度推定を使用する場合（コメントを解除）
# pipeline_real = create_pipeline(use_mock_depth=False)
# result_real = pipeline_real.process(image, custom_bbox=bbox, k=5)

## カスタム設定の使用

In [ ]:
# カスタム設定でパイプラインを作成
settings = ShadowboxSettings()

# クラスタリング設定
settings.clustering.min_k = 3
settings.clustering.max_k = 8

# レンダリング設定
settings.render.layer_thickness = 0.15
settings.render.layer_gap = 0.02

# カスタム設定でパイプラインを作成
# pipeline_custom = create_pipeline(settings, use_mock_depth=True)

print("カスタム設定:")
print(f"  クラスタリング: k={settings.clustering.min_k}-{settings.clustering.max_k}")
print(f"  レイヤー厚み: {settings.render.layer_thickness}")

## 終わりに

このノートブックでは、TCGカードからインタラクティブな3Dシャドーボックスを
作成する基本的なワークフローを紹介しました。

### 次のステップ
- カード画像のURLを変更して、自分のカードで試してみてください
- レイヤー数（k）を変更して、最適な見た目を探してみてください
- レンダリングオプションを調整して、好みの表示にカスタマイズしてください

### 参考リンク
- [GitHub Repository](https://github.com/2ROSSO/shadowbox-generator)
- [Depth Anything v2](https://huggingface.co/depth-anything)